In [4]:
import pandas as pd     
import jieba

In [5]:
df_eng = pd.read_csv('C:/Users/20693/Desktop/智慧养老政策.csv', encoding='gbk',index_col = 0)
df_eng.head()

,标题,时效性,效力位阶,制定机关,发文字号,公布日期,施行日期,批准部门,批准日期,原文链接
序号,,,,,,,,,,
地方规范性文件_672,黑龙江省人民政府办公厅关于推进养老服务发展的实施意见,现行有效,地方规范性文件,黑龙江省民政厅,NaN,NaN,NaN,NaN,NaN,https://www.pkulaw.com/lar/b18e6be8f51692a1cbd...
地方规范性文件_673,广西壮族自治区大数据发展局关于全面推进数字化转型发展的意见,现行有效,地方规范性文件,广西壮族自治区大数据发展局,NaN,NaN,NaN,NaN,NaN,https://www.pkulaw.com/lar/22f0c8beef6908e6866...
地方规范性文件_674,中共无锡市委、无锡市人民政府印发《＜关于加快推进数字经济高质量发展的实施意见＞＜关于加快推进...,现行有效,地方规范性文件,无锡市人民政府,NaN,NaN,NaN,NaN,NaN,https://www.pkulaw.com/lar/42b6a5f466eaa99be8e...
地方规范性文件_675,贵阳市民政局关于推进贵阳贵安养老服务高质量发展的实施意见,现行有效,地方规范性文件,贵阳市民政局,NaN,NaN,NaN,NaN,NaN,https://www.pkulaw.com/lar/e67d28d08fea8084a66...
地方规范性文件_676,廊坊市人民政府办公室关于加快推进养老服务体系建设的实施意见,现行有效,地方规范性文件,廊坊市人民政府,NaN,NaN,NaN,NaN,NaN,https://www.pkulaw.com/lar/909deb5c2273a94ef8b...


In [8]:
dlut = pd.read_excel('C:/Users/20693/Desktop/dlut-emotion-lexicon.xlsx', usecols=['词语', '词性种类', '情感分类', '强度'])
dlut.head()

,词语,词性种类,情感分类,强度
0,脏乱,adj,NN,7
1,糟报,adj,NN,5
2,早衰,adj,NE,5
3,责备,verb,NN,5
4,贼眼,noun,NN,5


In [10]:
Happy, Good, Surprise, Anger, Sad, Fear, Disgust  = [], [], [], [], [], [], []

for idx, row in dlut.iterrows():
    if row['情感分类'] in ['PA', 'PE']:
        Happy.append(row['词语'])
    if row['情感分类'] in ['PD', 'PH', 'PG', 'PB', 'PK']:
        Good.append(row['词语']) 
    if row['情感分类'] in ['PC']:
        Surprise.append(row['词语'])     
    if row['情感分类'] in ['NA']:
        Anger.append(row['词语'])    
    if row['情感分类'] in ['NB', 'NJ', 'NH', 'PF']:
        Sad.append(row['词语'])
    if row['情感分类'] in ['NI', 'NC', 'NG']:
        Fear.append(row['词语'])
    if row['情感分类'] in ['NE', 'ND', 'NN', 'NK', 'NL']:
        Disgust.append(row['词语'])
Positive = Happy + Good + Surprise
Negative = Anger + Sad + Fear + Disgust

In [13]:
emo_dlut = pd.DataFrame(columns=['length_dlut', 'positive_dlut', 'negative_dlut',
                                'anger_dlut', 'disgust_dlut', 'fear_dlut', 'good_dlut',
                                'sadness_dlut', 'surprise_dlut', 'happy_dlut'])

for dc in df_eng.index:
    positive, negative, anger, disgust, fear, sad, surprise, good, happy = 0, 0, 0, 0, 0, 0, 0, 0, 0
    wordlist = list(jieba.cut(df_eng['标题'][dc]))
    wordset = set(wordlist)
    wordfreq = []
    for word in wordset:
        freq = wordlist.count(word)
        if word in Positive:
            positive += freq
        if word in Negative:
            negative += freq
        if word in Anger:
            anger += freq
        if word in Disgust:
            disgust += freq
        if word in Fear:
            fear += freq
        if word in Sad:
            sad += freq
        if word in Surprise:
            surprise += freq
        if word in Good:
            good += freq
        if word in Happy:
            happy += freq
            
    emotion_info = {
        'length_dlut': len(wordlist),
        'positive_dlut': positive,
        'negative_dlut': negative,
        'anger_dlut': anger,
        'disgust_dlut': disgust,
        'fear_dlut': fear,
        'good_dlut': good,
        'sadness_dlut': sad,
        'surprise_dlut': surprise,
        'happy_dlut': happy
    }
    
    emo_info = pd.DataFrame([emotion_info])
    emo_dlut = pd.concat([emo_dlut, emo_info], ignore_index=True)
    
emo_dlut.head()

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\20693\AppData\Local\Temp\jieba.cache
Loading model cost 1.518 seconds.
Prefix dict has been built successfully.


,length_dlut,positive_dlut,negative_dlut,anger_dlut,disgust_dlut,fear_dlut,good_dlut,sadness_dlut,surprise_dlut,happy_dlut
0,11,3,0,0,0,0,3,0,0,0
1,12,3,0,0,0,0,3,0,0,0
2,46,10,0,0,0,0,10,0,0,0
3,13,3,0,0,0,0,3,0,0,0
4,12,3,0,0,0,0,3,0,0,0


In [15]:
df_eng = pd.concat([df_eng, emo_dlut], axis=1)
df_eng.head()

,标题,时效性,效力位阶,制定机关,发文字号,公布日期,施行日期,批准部门,批准日期,原文链接,length_dlut,positive_dlut,negative_dlut,anger_dlut,disgust_dlut,fear_dlut,good_dlut,sadness_dlut,surprise_dlut,happy_dlut
地方规范性文件_672,黑龙江省人民政府办公厅关于推进养老服务发展的实施意见,现行有效,地方规范性文件,黑龙江省民政厅,NaN,NaN,NaN,NaN,NaN,https://www.pkulaw.com/lar/b18e6be8f51692a1cbd...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
地方规范性文件_673,广西壮族自治区大数据发展局关于全面推进数字化转型发展的意见,现行有效,地方规范性文件,广西壮族自治区大数据发展局,NaN,NaN,NaN,NaN,NaN,https://www.pkulaw.com/lar/22f0c8beef6908e6866...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
地方规范性文件_674,中共无锡市委、无锡市人民政府印发《＜关于加快推进数字经济高质量发展的实施意见＞＜关于加快推进...,现行有效,地方规范性文件,无锡市人民政府,NaN,NaN,NaN,NaN,NaN,https://www.pkulaw.com/lar/42b6a5f466eaa99be8e...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
地方规范性文件_675,贵阳市民政局关于推进贵阳贵安养老服务高质量发展的实施意见,现行有效,地方规范性文件,贵阳市民政局,NaN,NaN,NaN,NaN,NaN,https://www.pkulaw.com/lar/e67d28d08fea8084a66...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
地方规范性文件_676,廊坊市人民政府办公室关于加快推进养老服务体系建设的实施意见,现行有效,地方规范性文件,廊坊市人民政府,NaN,NaN,NaN,NaN,NaN,https://www.pkulaw.com/lar/909deb5c2273a94ef8b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df_eng.columns

Index(['标题', '时效性', '效力位阶', '制定机关', '发文字号', '公布日期', '施行日期', '批准部门', '批准日期',
       '原文链接', 'length_dlut', 'positive_dlut', 'negative_dlut', 'anger_dlut',
       'disgust_dlut', 'fear_dlut', 'good_dlut', 'sadness_dlut',
       'surprise_dlut', 'happy_dlut'],
      dtype='object')